# An easy *mitoSpotter* running pipeline for markers <img src='./webui/static/logo.png' align="right" alt="" width="120" />

Welcome to the *mitoSpotter* marker analysis pipeline! This notebook provides a step-by-step guide to identifying mitochondrial and nuclear gene markers using Hidden Markov Models (HMMs). Whether you are new to bioinformatics or an experienced researcher, this pipeline is designed to be accessible, reproducible, and efficient.

All the data, scripts, and output from scripts are already in-place in our well-structured directory. Detailed file connection illustration is as follows.

NOTE: This notebook provides the exact same workflow and instrcution as the one in `README.md`, but this one is in a pure command-line format. So if you wanna replace the code or some arguments and run directly, we highly recommand you to follow this jupyter notebook version.

![All scripts documentation](./webui/static/All_scripts_documentation.jpg)

All the preprocessed dataset (e.g., output from 01-03 scripts have been deposited in out_dir directory, but if you want to run this part, you can still do it!).

## At a glimpse: Overall script structure

Below is the directory structure of the `scripts/` folder, which contains all Python scripts used in this pipeline. The flow is sequential and each script performs a specific task. The output of one script serves as the input for the next.

/mitoSpotter/scripts  
├── 01_from_gtf_extract_id.py  
├── 02_fasta_split_by_id.py  
├── 03_sequence2unit_nt.py  
├── 04_train_hmm_nt.py  
└── 05_decode_path_nt.py  

## Before we start



### Prerequisites

Before executing any cells in this notebook, please ensure the following:

1. **Environment Setup**:  
   You should have configured the conda or virtual environment as described in the main documentation. Also, ensure that the correct kernel is selected in your Jupyter environment.

2. **Download Required Data**:  
   The pipeline depends on two essential genomic files:
   - **CDS genome file**: [Homo_sapiens.GRCh38.cds.all.fa.gz](https://42basepairs.com/browse/web/ensembl/release-82/fasta/homo_sapiens/cds?file=Homo_sapiens.GRCh38.cds.all.fa.gz&preview=)
   - **GTF annotation file**: [Homo_sapiens.GRCh38.115.chr.gtf.gz](https://ftp.ensembl.org/pub/release-115/gtf/homo_sapiens/Homo_sapiens.GRCh38.115.chr.gtf.gz)  
   Download and place them in the `/mitoSpotter/data` directory. *Note: This download may take several minutes depending on your connection speed.*

3. **Import Necessary Packages**:  
   Run the cell below to import the required Python libraries. If any import fails, please check your environment installation.

In [ ]:
import sys
import runpy
from pathlib import Path
import itertools
import pandas as pd
from pathlib import Path
from IPython.display import display, Image

## STEP 1: EXTRACT NUCLEAR AND MITOCHONDRIA GENE ID

**Input**: GTF file  
**Output**: Two text files containing mitochondrial and nuclear gene IDs.  
**Key parameter**: `--protein_coding_only` ensures we only retain protein-coding genes, which are relevant for marker identification.

In [ ]:
sys.argv = [
    "scripts/01_from_gtf_extract_id.py",
    "--gtf", "data/Homo_sapiens.GRCh38.115.chr.gtf",
    "--outdir", "out_dir/01_ids",
    "--prefix", "human_protein_coding_marker_testing_",
    "--protein_coding_only",
]

runpy.run_path("scripts/01_from_gtf_extract_id.py", run_name="__main__")

# This equals to running 

# python scripts/01_from_gtf_extract_id.py \
#   --gtf data/Homo_sapiens.GRCh38.115.chr.gtf \
#   --outdir out_dir/01_ids \
#   --protein_coding_only \
#   --prefix human_protein_coding_

# at the terminal.

### Expected Output

![01 output](./webui/static/01_output.png)

## 02 SPLIT FASTA BY ID

In this step, we want to get the sequence with these ids.

**Input**: A complete *5'UTR + CDS + 3' UTR* genome sequence file (FASTA format, from the previous step)

**Output**: Two FASTA files containing mitochondrial and nuclear gene sequences

**Key parameters**:

--mito_ids: Path to file containing mitochondrial gene IDs

--nuc_ids: Path to file containing nuclear gene IDs

--prefix: Custom prefix for output filenames

In [ ]:
sys.argv = [
    "scripts/02_fasta_split_by_id.py",
    "--fasta", "data/Homo_sapiens.GRCh38.cds.all.fa",
    "--mito_ids", "out_dir/01_ids/human_protein_coding_marker_testing_ids_mito.txt",
    "--nuc_ids", "out_dir/01_ids/human_protein_coding_marker_testing_ids_nuclear.txt",
    "--outdir", "out_dir/02_split_fasta",
    "--prefix", "human_marker_testing_",
]

runpy.run_path("scripts/02_fasta_split_by_id.py", run_name="__main__")

### Expected Output

![02 output](./webui/static/02_output.png)

## 03 TOKENIZATION AND SPLIT INTO TRAINING AND HOLDOUT DATASETS

**Input**: Separated FASTA files for mitochondrial and nuclear genes (from the previous step)

**Output**:

Training dataset files (TSV format)

Holdout dataset files (TSV format)

**Key parameters**:

--mode: Tokenization mode (1nt, 2nt, or 3nt)

--train_frac: Fraction of data to use for training (default: 0.7)

Multiple mode combinations are processed automatically for comparative analysis



In [ ]:
# Define the parameter combinations
locations = ["nuclear", "mito"]
kinds = ["cds"]
modes = ["3nt", "2nt", "1nt"]

# Iterate through all combinations
for loc, kind, mode in itertools.product(locations, kinds, modes):
    sys.argv = [
        "scripts/03_sequence2unit_nt.py",
        "--fasta", f"out_dir/02_split_fasta/human_{loc}_{kind}.fa",
        "--mode", mode,
        "--train_tsv", f"out_dir/03_unit/train/human_marker_testing_{loc}_{mode}_train.tsv",
        "--holdout_tsv", f"out_dir/03_unit/holdout/human_marker_testing_{loc}_{mode}_holdout.tsv",
        "--train_frac", "0.7",
    ]
    
    print(f"Running: {loc} {kind} {mode}")
    runpy.run_path("scripts/03_sequence2unit_nt.py", run_name="__main__")

### Expected Output

![03 output](./webui/static/03_output.png)

## 04 HMM TRAINING

Yeahh! Finally comes into training phase! Let me take 3-nt level for example.

![04 param](./webui/static/04_param.png)

### EM run

In [ ]:
sys.argv = [
    "scripts/04_train_hmm_nt.py",
    "--nuclear_nt_tsv", "out_dir/03_unit/train/human_marker_testing_nuclear_3nt_train.tsv",
    "--mito_nt_tsv", "out_dir/03_unit/train/human_marker_testing_mito_3nt_train.tsv",
    "--ngram", "3",
    "--train_method", "em",
    "--learn", "et", # use et to learn both emission and transition, essential!
    "--n_em_iter", "20",
    "--out_model_json", "out_dir/04_model/human_marker_testing_3nt_model_em.json",
    "--out_vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_em.json",
    "--out_states_json", "out_dir/04_model/human_marker_testing_3nt_states_em.json",
    "--sample", "0.0001", # downsample for faster training, expected to take less than 1 minute
    "--track_memory",
    "--n_workers", "2", # if you use mac or linux, add this, otherwise remove this line
]

runpy.run_path("scripts/04_train_hmm_nt.py", run_name="__main__")

### Viterbi

In [ ]:
sys.argv = [
    "scripts/04_train_hmm_nt.py",
    "--nuclear_nt_tsv", "out_dir/03_unit/train/human_marker_testing_nuclear_3nt_train.tsv",
    "--mito_nt_tsv", "out_dir/03_unit/train/human_marker_testing_mito_3nt_train.tsv",
    "--ngram", "3",
    "--train_method", "viterbi",
    "--n_viterbi_iter", "20",
    "--out_model_json", "out_dir/04_model/human_marker_testing_3nt_model_viterbi.json",
    "--out_vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_viterbi.json",
    "--out_states_json", "out_dir/04_model/human_marker_testing_3nt_states_viterbi.json",
    "--sample", "0.0001", # downsample for faster training, expected to take less than 1 minute
    "--track_memory",
    "--n_workers", "2", # if you use mac or linux, add this, otherwise remove this line
]

runpy.run_path("scripts/04_train_hmm_nt.py", run_name="__main__")

### Hybrid

In [ ]:
# Let's make em 0.5 and viterbi 0.5 (1:1)

sys.argv = [
    "scripts/04_train_hmm_nt.py",
    "--nuclear_nt_tsv", "out_dir/03_unit/train/human_marker_testing_nuclear_3nt_train.tsv",
    "--mito_nt_tsv", "out_dir/03_unit/train/human_marker_testing_mito_3nt_train.tsv",
    "--ngram", "3",
    "--train_method", "hybrid",
    "--n_em_iter", "10",
    "--n_viterbi_iter", "10",
    "--learn", "et", # use et to learn both emission and transition, essential!
    "--out_model_json", "out_dir/04_model/human_marker_testing_3nt_model_hybird.json",
    "--out_vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_hybird.json",
    "--out_states_json", "out_dir/04_model/human_marker_testing_3nt_states_hybird.json",
    "--sample", "0.0001", # downsample for faster training, expected to take less than 1 minute
    "--track_memory",
    "--n_workers", "2", # if you use mac or linux, add this, otherwise remove this line
]

runpy.run_path("scripts/04_train_hmm_nt.py", run_name="__main__")

## 05 SEQUENCE DECODING

![05 param](./webui/static/05_param.png)

### FASTA

In [ ]:
sys.argv = [
  "05_decode_path_nt.py",

  # Load the model!
  "--model_json", "out_dir/04_model/human_marker_testing_3nt_model_em.json",
  "--vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_em.json",
  "--states_json", "out_dir/04_model/human_marker_testing_3nt_states_em.json",

  # correspondingly, use the "3-nt" mode
  "--ngram", "3",

  # An example file of fasta sequences (from a becteria which has a relatively small phylogenetic distance to mitochondria genes)
  "--fasta", "./webui/static/Rickettsia_prowazekii_str_Madrid_E.fa",

  # don't decode too short sequences (<10)
  "--min_len", "10",

  # Write output
  "--out_tsv", "out_dir/05_res/human_marker_testing_fasta_res.tsv",

  "--plotting", # plot the results! the plots will be saved in the same directory where you run the script
  "--track_memory",
]

runpy.run_path("scripts/05_decode_path_nt.py", run_name="__main__")

### ARGUMENT SEQUENCE PASSING

In [ ]:
sys.argv = [
  "05_decode_path_nt.py",

  # Load the model!
  "--model_json", "out_dir/04_model/human_marker_testing_3nt_model_em.json",
  "--vocab_json", "out_dir/04_model/human_marker_testing_3nt_vocab_em.json",
  "--states_json", "out_dir/04_model/human_marker_testing_3nt_states_em.json",

  # correspondingly, use the "3-nt" mode
  "--ngram", "3",

  # don't decode too short sequences (<10)
  "--min_len", "10",

  # The first sequence
  "--seq", "ATACCCATGGCCAACCTCCTACTCCTCATTGTACCCATTCTAATCGCAATGGCATTCCTAATGCTTACCGAACGAAAAATTCTAGGCTATATACAACTACGCAAAGGCCCCAACGTTGTAGGCCCCTACGGGCTACTACAACCCTTCGCTGACGCCATAAAACTCTTCACCAAAGAGCCCCTAAAACCCGCCACATCTACCATCACCCTCTACATCACCGCCCCGACCTTAGCTCTCACCATCGCTCTTCTACTATGAACCCCCCTCCCCATACCCAACCCCCTGGTCAACCTCAACCTAGGCCTCCTATTTATTCTAGCCACCTCTAGCCTAGCCGTTTACTCAATCCTCTGATCAGGGTGAGCATCAAACTCAAACTACGCCCTGATCGGCGCACTGCGAGCAGTAGCCCAAACAATCTCATATGAAGTCA",
  "--seq_id", "marker_test_1",

  # The second sequence
  "--seq",
  "GTCACCCTAGCCATCATTCTACTATCAACATTACTAATAAGTGGCTCCTTTAACCTCTCCACCCTTATCACAACACAAGAACACCTCTGATTACTCCTGCCATCATGACCCTTGGCCATAATATGATTTATCTCCACACTAGCAGAGACCAACCGAACCCCCTTCGACCTTGCCGAAGGGGAGTCCGAACTAGTCTCAGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTCATAGCCGAATACACAAACATTATTATAATAAACACCCTCACCACTACAATCTTCCTAGGAACAACATATGACGCACTCTCCCCTGAACTCTACACAACATATTTTGTCACCAAGACCCTACTTCTAACCTCCCTGTTCTTATGAATTCGAACAGCATACCCCCGATTCCGCTACGACCAACTCATACACCTCCTATGAAAAAACTTCCTACCACTCACCCTAGCATTACTTATATGATATGTCTCCATACCCATTACAATCTCCAGCATTCCCCCTCAAACC",
  "--seq_id", "marker_test_2",

  # Write output
  "--out_tsv", "out_dir/05_res/human_marker_testing_arg_res.tsv",

  "--plotting", # plot the results! the plots will be saved in the same directory where you run the script
  "--track_memory",
]

runpy.run_path("scripts/05_decode_path_nt.py", run_name="__main__")

## Visualization of Results

The decoding step also generates several informative plots (if you pass "--plotting") to help you interpret prediction results from the model and the compositional features of your sequences:

- **Classification Counts**: Shows the proportion of sequences classified as nuclear vs. mitochondrial.  
  ![classification_counts](./webui/static/Plot_example/classification_counts.png)

- **GC Content per Sequence**: Shows the GC content proportion across sequences, which often differs between nuclear and mitochondrial genes.
  ![gc_content_stacked_bar](./webui/static/Plot_example/gc_content_stacked_bar.png)

- **Log-Likelihood Distribution**: Displays the log-likelihood scores for all decoded sequences; higher values indicate better model fit.
  ![loglikelihood_distribution](./webui/static/Plot_example/loglikelihood_distribution.png)

- **Hidden State Proportions**: Illustrates the proportion of "time" each sequence spends in the “nuclear” or “mitochondrial” hidden state.
  ![state_proportions_stacked_bar](./webui/static/Plot_example/state_proportions_stacked_bar.png)